In [1]:
from docplex.mp.model import Model
from collections import namedtuple
import pandas as pd
import numpy as np
import math
import random
import copy
import time
import os
from pyswarm import pso
import pyswarms as ps

In [30]:
#  ------------------BILEVEL: PYSWARM AND CPLEX  -------------
max_objective = 0
max_values =[]
optimal_prices = []
iteration_reqd =0
start = time.time()
# STEP1 : INITIALIZATION 
# PRICES = min_list
PRICES = lb
# PRICES = price_initialization( K, J, R_K , PRICES)
x =[]
   
# ---------LOOP ---
for it_count in range(0, 10 ):
#     print("\n\n\n ITERATION NUMBER :  ", it_count )
    # Read only for the first iteration   
#     if iter ==0:
#         K,J , R_K, BUDGETS = read_instance()
        
    
# STEP2: SOLVE LOWER LEVEL BY CPLEX  

    x.append( master_lower_level(K, J ,  PRICES, R_K, BUDGETS, S_K  ) )                 # SOLVES CPLEX 
#     x.append( lower_level_optimal(K,J, PRICES , BUDGETS , R_K, S_K ) )            # SOLVES USING PYTHON FUNCTION 
#     print("\n Assignment at Iteration , Lower and Upper Objectives \n",  x[it_count] )
#     print('Assignment after lower level', prod_number(x[it_count][0], K , J ) )
# ---------Evaluation of Objective Function 
    objective = x[it_count][2]
    #print("Objective at iteration : \n" ,   objective)
    
    if objective > max_objective:
        max_objective = copy.deepcopy(objective)  
        max_values = copy.deepcopy(x[it_count]) 
        optimal_prices = copy.deepcopy(PRICES)
        iteration_reqd = it_count        
    
#STEP3: SOLVE UPPER LEVEL 
#     print('Price before PSO :', PRICES)
#  USING PYSWARM -----------------------------------------------------------------------------
    xopt, fopt = pso(banana , lb, ub , args = (x, it_count)   ,maxiter=100  , debug=False )  # Returns Numpy Array 
    PRICES = xopt 
#     print("New Prices Before Round off: "   , xopt,  fopt  )
    price_roundoff(PRICES)
#     print("Prices after PSO   : "   , PRICES)
    
    # Pertubation strategy after every 100 iteration
    if it_count % 10  == 0:
#         print(it_count)
#         PRICES =  rearrange_reservation( x[it_count][0] , K, J, it_count, R_K, BUDGETS, S_K, PRICES )                # Price is set to the customer willing to pay for it 
        PRICES = price_pertubation( K, J, R_K , PRICES)
        price_roundoff(PRICES)
#         print('Price after Pertubation', PRICES)
#   if it_count > 100:
#         if x[it_count-5][2] == x[it_count][2]:
#             break
#print(xopt,fopt) 

end = time.time()
print('Time Elapsed' , end - start)
print('\n \n Iterations completed',it_count)
print('\n Maximum Objective obtained \n',max_objective, max_values[2]   )
print('\n Lower Level Objective \n',max_values[1])
print('\n Assignment of products \n' , max_values[0] )
print('Prices of products ', optimal_prices )
# print('Numbered Prod Purchased ',  prod_number(max_values[0], K , J ))  use this if lower level solved using cplex 
 

Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Stopping search: maximum iterations reached --> 100
Time Elapsed 5.981148958206177

 
 Iterations completed 9

 Maximum Objective obtained 
 128.0 128.0

 Lower Level Objective 
 70.8

 Assignment of products 
 [1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1.0, 0, 0]


In [ ]:
print('Numbered Prod Purchased ',  prod_number(max_values[0], K , J )) 

In [54]:
print(iteration_reqd)

1390


In [8]:
# -------------CPLEX LOWER LEVEL ------------------
def master_lower_level( K, J , PRICES, R_K, BUDGETS, S_K  , **kwargs):  
   
     
    #iterations +=1 
    #     ----- Result Variable (Variables to store) ----------------
    Cust_Ass =[]
    lower_objective = 0
    
    mdl = Model(name='lowerlevel' )

    weight =0.8          # Weight 
   
    R = range(0,K)
    # ------ decision variables ---
    x  = mdl.binary_var_matrix(K,J ,"cust_assign", key_format ="_%s")
     
    # --------constraints 1---
    for i in R:  
        mdl.add_constraint( mdl.sum(  x[i,j] for j in R )  <= 1   )

    # --------- constraints2 ---
    for i in R:
        mdl.add_constraint( mdl.sum(   x[i,j]* PRICES[j] for j in R  )  <= BUDGETS[ i]    )
   
    # --------- constraints3 ---
    for k in R:
        for j in R:
            mdl.add_constraint(    x[k,j] <=   S_K[k][j]  )
  
    # ------------ objective ---         
    mdl.maximize( mdl.sum(   weight*  x[i,j ] *S_K[i][j] +  x[i,j ] *(1-weight)* (R_K[i][j] - PRICES[j]  )     for i in R for j in R) )      
    
    # ------------ KPI  ---   
    mdl.add_kpi(  mdl.sum( x[i,j ]*PRICES[j]      for i in R for j in R )  , publish_name = 'UpperObjective')  

 
    msol = mdl.solve()                 #log_output=True              # msol is a data structure of the class Solve Solution
    # ----- Retrieving values of decision variables 
    for i in range(0,K):
            for j in range(0,K):
                    ld = msol.get_value(x[i,j ])
                    Cust_Ass.append(ld) 
                    
    lower_objective = msol.get_objective_value()                # Retrieves obj value 
    upper_objective = msol.kpi_value_by_name('UpperObjective')
    
    return Cust_Ass , lower_objective ,upper_objective

In [9]:
def read_instance():
    os.chdir("D:\AA MASC\C Codes\FINAL\INSTANCES")
    with open("test_10k_10ta_1.txt", 'r' ) as file:         # Hardcoded file name 
        lines = file.readlines()
        text_file = file.read()
        file.close()
        
    i=1
    TF = []
    Rdash = []
    RR = []

    for line in lines:
        if i==1:
            J = int (line.split()[2])      # No. of Products 
            i+=1
#             print(J)
            continue

        if i==2:
            K = int (line.split()[2] )     # No. of Customers 
            i+=1
#             print(K)
            continue

        if i==3:
            i+=1
            continue 

        if i == 4:                       # i = 4 , denotes 4th line 
    #         print(line)
            for iter in range(0,K):
#                 print(iter)
#                 print(line.split()[iter+3 ])
                TF.append( int ( line.split()[ iter+3 ] ) ) 
#                 print(TF)
            i+=1
            continue

        if ( 4 < i < 5+ J):
            for iter in range(0,K):
#                 print('iter' , iter)
                if i == 5:
#                     print(line.split()[iter+3])
                    Rdash.append( int ( line.split()[ iter+3  ] ) ) 
                else:
#                     print(line.split()[iter])
                    Rdash.append( int ( line.split()[ iter  ] ) ) 
            i+=1
            RR.append(Rdash)
            Rdash=[]        
            continue
        
#         print(RR)
        NestList = []
        R_K =[] 
        BUDGETS = []
        
        for k in range(0,K):                   # For each customer 
            for j in range(0,J):   
#                 print( TF[j])
                NestList.append( TF[k] -  RR[j][k]) 
            R_K.append(NestList)
            BUDGETS.append(max(NestList))
            NestList=[]
        
        
         
        return K,J , R_K, BUDGETS

In [10]:
K,J , R_K, BUDGETS = read_instance()

In [11]:
S_K = [[10, 9, 0, 8, 7, 6, 5, 0, 4, 3], [10, 9, 0, 8, 7, 0, 0, 6, 0, 5], [10, 9, 8, 0, 7, 6, 0, 5, 0, 4], [0, 0, 0, 0, 10, 9, 0, 0, 8, 7], [10, 9, 8, 0, 7, 0, 6, 5, 4, 3], [10, 0, 9, 8, 7, 0, 6, 5, 0, 4], [0, 10, 9, 8, 7, 6, 5, 0, 0, 0], [10, 9, 0, 8, 0, 0, 7, 0, 0, 6], [0, 10, 9, 0, 0, 8, 7, 6, 5, 4], [0, 0, 10, 9, 0, 0, 8, 7, 6, 5]]


In [12]:
def price_initialization( no_prod , no_cust,  R_K , PRICES): 
    product_prices_offered = []
    for i in range(no_prod):
        prd_nest = []
        for j in range(no_cust):
#             prd_nest.append( cust_list[j] for cust_list in R_K[j][i] ) 
            res_price = R_K[j][i] 
            prd_nest.append(res_price)
        product_prices_offered.append(prd_nest)
        PRICES[i] = min ( product_prices_offered[i] )  
#     print(product_prices_offered)
#     print(PRICES)
    
    return PRICES 


In [13]:
#-----------DONT RUN  (RUN ONLY ONCE )----------------------------
def Rank_Generator():
    S_K =  np.random.randint(0, 10, size=(K, J)).tolist()
    return S_K
# print(S_K)
Ranked_List = Rank_Generator()

In [28]:
# -------------Computing Upper Bound 
max_list = [0] * 10

for i in range(0,J):
    for j in range(0,K):
#         if max_list[i] < R_K[j][i]:
            max_list[i] = max(BUDGETS)

ub = max_list

In [18]:
# ---------- Computing Lower Bound 
min_list = [max(BUDGETS)] * 10

for i in range(0,J):
    for j in range(0,K):
        if min_list[i] > R_K[j][i]:
            min_list[i] = R_K[j][i]
            
lb = min_list

In [27]:
# -- PYSWARM for UPPER LEVEL--------
def banana(p, cus_ass , it_count):
    #print('in pYswarm' , x[iter][0])
    obj = 0
    i=0 
    No_Prod = len(max_list)
#     print('Assignment Receive in PYSWARM', cus_ass[it_count])  
    for value in cus_ass[it_count][0] :  
       
        obj = obj + p[i] * value 
        
        i += 1
        if (i==No_Prod):
            i=0
    return  -obj
 
# lb =  min_list
# ub = max_list

In [20]:
# ---------Rearrange Logic for RESERVATION  --------- 
def rearrange_reservation( numbered_list, no_prod , no_cust , call_iter , R_K, BUDGETS, S_K , PRICES):
    i = 0
    for val in numbered_list:
        if val != 0:
#             print(val-1  ,PRICES[val-1],  R_K[i][val-1]   )
            if (PRICES[val-1] <  R_K[i][val-1]):
                PRICES[val-1] =  R_K[i][val-1]
            i = i+1 
    return PRICES 

In [21]:
def price_pertubation( no_prod , no_cust,  R_K , PRICES): 
    product_prices_offered = []
    for i in range(no_prod):
        prd_nest = []
        for j in range(no_cust):
#             prd_nest.append( cust_list[j] for cust_list in R_K[j][i] ) 
            res_price = R_K[j][i] 
            prd_nest.append(res_price)
        product_prices_offered.append(prd_nest)
        PRICES[i] = min ( max(product_prices_offered[i]) *  np.random.uniform(1,1.25) , max(BUDGETS)  )
#     print(product_prices_offered)
#     print(PRICES)
    
    return PRICES 
# price_pertubation( K, J, R_K , PRICES)

In [22]:
# ---------FUNCTION TO ROUND OFF PRICES FROM PYSWARM ----------
def price_roundoff(PRICES):
    index = 0
    for price in PRICES:
        if (price.is_integer() != True) :
            PRICES[index] = math.ceil(price)
        index +=1

In [23]:
# ----------Funtion to Compute , Assignment 
def prod_number(cust_list, no_prod , no_cust ):
    numbered_list = []
    for i in range(no_prod):
        flag = 0
        j = 0
        for j in range(no_cust ):  
            if ( cust_list[int(str(i) + str(j))]  >  0 ) :  
                numbered_list.append(j+1)
                flag=1
            j += 1
        if flag ==0:
            numbered_list.append(0)
            
    return numbered_list

# This list contains actual index i.e. Product1, Prod 2 NOT python indices from 0,
# A value of 0 indicates customer did not made any purchase 

In [110]:

##################################################################################33333
# -----------CHECKING LOWER LEVEL IS OPTIMAL 
##################################################################################33333
 

def lower_optimality_check(K,J, PRICES , BUDGETS , R_K, S_K ):
    weight = 0.8
    Best_Cust_Ass = []
    for i in range(K):         # For each customer 
        opt_val = 0
        opt_sel= 0
        for j in range(J):
            if ( S_K[i][j] == 0 or BUDGETS[i] < PRICES[j] ):
                obj = -1 
            else:
#                 obj = round( weight*S_K[i][j] + (1-weight)* ( R_K[i][j] - PRICES[j]), 2)  
                obj = weight*S_K[i][j] + (1-weight)* ( R_K[i][j] - PRICES[j])
            if obj > opt_val:
                opt_val = obj
                opt_sel = j+1
#                 print('Customer %d optimal selection is'   %(i+1), opt_sel)
        Best_Cust_Ass.append(opt_sel)
 
    return Best_Cust_Ass

print('From Heurestic', prod_number( max_values[0], K , J ) )
a = lower_optimality_check(K,J, optimal_prices , BUDGETS , R_K, S_K )
print('Recheck function', a)
if (a == prod_number( max_values[0], K , J )):
    print('Lower Level is Optimal ')
else:
    print('Not Optimal')

From Heurestic [4, 4, 4, 1, 7, 9, 5, 1, 1, 7]
Recheck function [4, 4, 4, 1, 7, 9, 5, 1, 1, 7]
Lower Level is Optimal 


In [31]:
# --------Visualizing and Comparing the Result ---------

def obj_evaluation(K,J, PRICES, BUDGETS, R_K, S_K):
    Obj_list = []
    weight = 0.8
    for i in range(K):         # For each customer 
#         print("Customer is %d"%(i+1))
        Nest_O = []
        for j in range(J):
#             print("Product is %d"%(j+1))
#             print("Reservation: %d and Rank: %d"%( R_K[i][j], S_K[i][j]))                   or BUDGETS[i] < PRICES[j]
            if (S_K[i][j] == 0 or PRICES[j]>BUDGETS[i] ):
#                 print("Customer %d , is unable to purchase Prod:%d as Budget is %d and Price is %d"%( i+1,j+1, BUDGETS[i], PRICES[j] ))
                obj = -1 
            else:
#                 obj = round( weight*S_K[i][j] + (1-weight)* ( R_K[i][j] - PRICES[j]), 2)  
                obj = weight*S_K[i][j] + (1-weight)* ( R_K[i][j] - PRICES[j])      
            Nest_O.append(obj)
        Obj_list.append(Nest_O)        
    return Obj_list

new_list = obj_evaluation(K,J, optimal_prices ,BUDGETS,  R_K, S_K)

columns=["Cust1", "Cust2","Cust3","Cust4","Cust5","Cust6","Cust7", "Cust8", "Cust9", "Cust10"]
products = ["Prod1","Prod2","Prod3","Prod4","Prod5","Prod6","Prod7","Prod8","Prod9","Prod10" ]
df =pd.DataFrame(new_list, columns=products, index= columns   )
df['Budgets'] = BUDGETS
df['Selection'] = prod_number(max_values[0], K , J )  #if cplex used
# df['Selection'] = max_values[0]
pric =[]
pric = copy.deepcopy(optimal_prices.tolist())
pric.append(0)
pric.append(0)
df.loc['Prices'] = pric

df_RK = pd.DataFrame(R_K, columns=products, index= columns  )
# df_RK['Budgets'] = BUDGETS
df_SK = pd.DataFrame(S_K, columns=products ,index= columns )
df_concat = pd.concat([df_RK,df_SK] , axis = 1)

# df

In [15]:
# FUNCTION TO SOLVE LOWER LEVEL TO OPTIMALITY 
def lower_level_optimal(K,J, PRICES , BUDGETS , R_K, S_K ):
    weight = 0.8
    Best_Cust_Ass = []
    lower_obj = 0
    upper_obj = 0
    for i in range(K):         # For each customer 
        opt_val = 0
        opt_sel= 0
        prod_selection_flag = 0
        for j in range(J):
            if ( S_K[i][j] == 0 or BUDGETS[i] < PRICES[j] ):
                obj = -1 
            else:
                obj = weight*S_K[i][j] + (1-weight)* ( R_K[i][j] - PRICES[j])
            if obj >= opt_val:          # Checks whether the current obj is greater than equal to the previous best
                if obj == opt_val:    # If current obj is = to previous best 
                    if ( PRICES[j] > PRICES[opt_sel] ):   # Checking if PRICE of current obj PRODUCT is > than price of previous best
                    # IF PRICE more than replace as it contributes to leader's objective better 
                        opt_val = obj
                        opt_sel = j+1
                        prod_selection_flag = 1
                    
                elif (obj > opt_val): # (automatically satisfied) else strictly greater than simply replace 
                    opt_val = obj
                    opt_sel = j+1
                    prod_selection_flag = 1
#         print('Customer %d optimal selection is'   %(i+1), opt_sel ,prod_selection_flag )
        Best_Cust_Ass.append(opt_sel)
        lower_obj = lower_obj + opt_val 
        upper_obj = upper_obj + PRICES[opt_sel-1] * prod_selection_flag
        
    return Best_Cust_Ass,lower_obj , upper_obj



In [67]:
# COMPARING TIMES , CPLEX vs PYTHON RESULT 

print(PRICES)
start = time.time()
cplex_result =  master_lower_level(K, J ,  PRICES, R_K, BUDGETS, S_K  )  
end = time.time()
print('Time is :', end - start )
print('Cust Selection ',  prod_number(cplex_result[0], K , J ))
print( 'Lower Obj', cplex_result[1])
print( 'Upper Obj', cplex_result[2])
start = time.time()
python_result = lower_level_optimal(K,J, PRICES , BUDGETS , R_K, S_K )
end = time.time()
print('Time is :', end - start )
print('Cust Selection ',  python_result[0] )
print( 'Lower Obj', python_result[1])
print( 'Upper Obj', python_result[2])

[10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
Time is : 0.022939205169677734
Cust Selection  [1, 2, 2, 1, 1, 1, 1, 4, 2, 1]
Lower Obj 94.0
Upper Obj 100.0
Time is : 0.0
Cust Selection  [1, 2, 2, 1, 1, 1, 1, 4, 2, 1]
Lower Obj 94.0
Upper Obj 100


In [32]:
# --PYSYWARM TIME  -------
start = time.time()
xopt, fopt = pso(banana , lb, ub , args = (x, it_count)   ,maxiter=100  , debug=False )
end = time.time()
print(end - start)

Stopping search: maximum iterations reached --> 100
0.5674848556518555


In [66]:
# CHECKING IF FOR DIFFERENT ASSIGNMENTS, PSO RETURNS DIFFERENT PRICES 
x = [ ]
PRICES = [10]*10
x.append( lower_level_optimal(K,J, PRICES , BUDGETS , R_K, S_K ) ) 
print(x)

[([1, 2, 2, 1, 1, 1, 1, 4, 2, 1], 94.0, 100)]


In [ ]:
xopt, fopt = pso(banana , lb, ub , args = (x, 0)   ,maxiter=100  , debug=False )  # Returns Numpy Array 
PRICES = xopt 